# Loading/joining/cleaning Data (SK, 23 11 29)

### Requirements to run notebook:
cost_living_w_codes.csv,
Gender Pay Gap.csv,
country_codes.sql (initialised as a DB in MySQL workbench),
output_gbp_salaries_23-11-29_10-55.csv # or your equivilent file

NB: I have tended to create a backup .csv copy of every DF created: these can be removed once happy with the code

In [16]:
# pip install sqlalchemy # if required

In [17]:
import pandas as pd
import matplotlib as plt
import numpy as np

Load the main .csv file which has all the combined data from our various API calls:  
country codes, local currency code, salaries in local currency (25th/50th/75th), conversion rate to gbp & gbp converted salaries.  
!! The final API file will have a timestamped (so unique) name in the format output_gbp_salaries_{timestamp}. Insert the filename or filepath

In [18]:
# Load main csv with salary data 
api_sal_df = pd.read_csv("output_gbp_salaries_23-11-29_10-55.csv")

### Merge salaries API data with country_codes data

The csv is difficult to work with at this point because it has only country codes, not country names.
We also want the area and population etc data which is stored in a separate country_codes.sql file.

You will need to have run country_codes.sql in MySQL to have created the database countries_db and the table country_codes  
You must also supply your MySQL login credentials below.

In [19]:
# mysql.connector or pymysql don't work with Jupyter / MySQl

In [20]:
# country_codes.sql information is in an .sql table. Need to convert sql > db in order for pandas to turn into into DF. Then convert to csv
from sqlalchemy import create_engine

# MySQl database connection details
username = 'root'
password = 'q%TjVDRGLK^Ak7s$5LYs' # complete this
host = 'localhost'
database = 'countries_db'

# Creates a database engine
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}")

# The SQL query to get all info from table named country_codes
query = "SELECT * FROM country_codes"  

# Use Pandas to load data into a DataFrame
countries_df = pd.read_sql_query(query, engine)

In [21]:
print(countries_df.head())

  iso_alpha2 iso_alpha3  iso_numeric fips               country  \
0         AD        AND           20   AN               Andorra   
1         AE        ARE          784   AE  United Arab Emirates   
2         AF        AFG            4   AF           Afghanistan   
3         AG        ATG           28   AC   Antigua and Barbuda   
4         AI        AIA          660   AV              Anguilla   

            capital  area_km2  population continent  
0  Andorra la Vella     468.0     77006.0        EU  
1         Abu Dhabi   82880.0   9630960.0        AS  
2             Kabul  647500.0  37172400.0        AS  
3        St. John's     443.0     96286.0        NA  
4        The Valley     102.0     13254.0        NA  


In [22]:
countries_df.to_csv("country_codes.csv", encoding='utf-8', index=False) # intermediate backup of DF to .csv file

Now join our main API salaries DF to DF made from the country_codes.sql file 

In [23]:
# Outer join api_sal_df to country_codes df on iso_alpha2
biggie_dfv1 = pd.merge(api_sal_df, countries_df, on='iso_alpha2', how='inner') # This is an inner join as we don't need info about countries for which we have no salary data (that being the focus of our data analysis)
biggie_dfv1.to_csv("sal_and_country.csv", index=False) # intermediate backup of DF to .csv file

In [24]:
biggie_dfv1.head()

,job_id,job_title,salary_percentiles_percentile_25,salary_percentiles_percentile_50,salary_percentiles_percentile_75,iso_alpha2,currency_code,local_to_gbp_rates,gbp_converted_25th,gbp_converted_50th,gbp_converted_75th,iso_alpha3,iso_numeric,fips,country,capital,area_km2,population,continent
0,ACCOUNT-MANAGER,Account Manager,4319.733426,5408.352120,6771.314286,AF,AFN,89.507995,48.260867,60.423118,75.650385,AFG,4,AF,Afghanistan,Kabul,647500.0,37172400.0,AS
1,ACCOUNTANT,Accountant,3447.692006,4094.865932,4863.522313,AF,AFN,89.507995,38.518258,45.748605,54.336178,AFG,4,AF,Afghanistan,Kabul,647500.0,37172400.0,AS
2,ADMINISTRATIVE-ASSISTANT,Administrative Assistant,2690.999854,3240.842667,3903.032984,AF,AFN,89.507995,30.064352,36.207298,43.605412,AFG,4,AF,Afghanistan,Kabul,647500.0,37172400.0,AS
3,ARCHITECT,Architect,5881.167914,7282.397219,9017.479186,AF,AFN,89.507995,65.705504,81.360299,100.744958,AFG,4,AF,Afghanistan,Kabul,647500.0,37172400.0,AS
4,ATTORNEY,Attorney,4705.830358,6355.070075,8582.314403,AF,AFN,89.507995,52.574414,71.000027,95.883216,AFG,4,AF,Afghanistan,Kabul,647500.0,37172400.0,AS


### Join gender pay data (column) with Biggie DF

In [25]:
# Read in the gender_pay_parity csv column
gender_df = pd.read_csv("Gender Pay Gap.csv")
gender_df.head(10)

,Country,Gender_Pay_Parity
0,Iceland,0.912
1,Norway,0.879
2,Finland,0.863
3,New Zealand,0.856
4,Sweden,0.815
5,Germany,0.815
6,Nicaragua,0.811
7,Namibia,0.802
8,Lithuania,0.800
9,Belgium,0.796


In [26]:
# Cleaning: rename Country to country / Gender_Pay_Parity gender_pay_parity to facilitate merge
gender_df.rename(columns={'Country': 'country', 'Gender_Pay_Parity':'gender_pay_parity'}, inplace=True)
gender_df.head(10)

,country,gender_pay_parity
0,Iceland,0.912
1,Norway,0.879
2,Finland,0.863
3,New Zealand,0.856
4,Sweden,0.815
5,Germany,0.815
6,Nicaragua,0.811
7,Namibia,0.802
8,Lithuania,0.800
9,Belgium,0.796


In [27]:
# join main salary_country DF with gender_gap column
biggie_dfv2 = pd.merge(gender_df, biggie_dfv1, on='country', how='outer') # ! Important, this is an outer join because we are joining on the 'country'(name) column rather than a controlled, standardised column like iso_alpha2. If this wasn't an outer join, we may miss data which doesn't match due to different spellings of names
biggie_dfv2.to_csv("sal_and_country_and_gender.csv", index=False) # intermediate backup of DF to .csv file
biggie_dfv2.head(10) 

,country,gender_pay_parity,job_id,job_title,salary_percentiles_percentile_25,salary_percentiles_percentile_50,salary_percentiles_percentile_75,iso_alpha2,currency_code,local_to_gbp_rates,gbp_converted_25th,gbp_converted_50th,gbp_converted_75th,iso_alpha3,iso_numeric,fips,capital,area_km2,population,continent
0,Iceland,0.912,ACCOUNT-MANAGER,Account Manager,61840.936200,77425.570997,96937.714928,IS,ISK,173.705482,356.010274,445.729001,558.057891,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
1,Iceland,0.912,ACCOUNTANT,Accountant,49358.448651,58618.849456,69616.643258,IS,ISK,173.705482,284.150207,337.461137,400.774014,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
2,Iceland,0.912,ADMINISTRATIVE-ASSISTANT,Administrative Assistant,38526.746533,46398.138724,55877.733542,IS,ISK,173.705482,221.793498,267.108085,321.680887,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
3,Iceland,0.912,ARCHITECT,Architect,84190.060978,104250.040552,129089.714734,IS,ISK,173.705482,484.671295,600.154004,743.152797,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
4,Iceland,0.912,ATTORNEY,Attorney,67367.124508,90974.341612,122854.150184,IS,ISK,173.705482,387.823825,523.727522,707.255458,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
5,Iceland,0.912,BUSINESS-ANALYST,Business Analyst,67652.687270,81155.074586,97352.321052,IS,ISK,173.705482,389.467773,467.199271,560.444725,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
6,Iceland,0.912,BUSINESS-DEVELOPMENT,Business Development,61002.216603,80818.868444,107072.986199,IS,ISK,173.705482,351.181873,465.263776,616.405337,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
7,Iceland,0.912,C-LEVEL-EXECUTIVE,C Level Executive,111662.206785,167410.372738,250991.214551,IS,ISK,173.705482,642.824887,963.759870,1444.923969,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
8,Iceland,0.912,CASHIER,Cashier,17880.934602,25050.371002,35094.423267,IS,ISK,173.705482,102.938229,144.211747,202.034057,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU
9,Iceland,0.912,CHEF,Chef,46041.508387,56551.727330,69461.187872,IS,ISK,173.705482,265.055011,325.560982,399.879077,ISL,352.0,IC,Reykjavik,103000.0,353574.0,EU


### Join cost of living data with Biggie DF

Now join our growing DF with the columns from cost_living_w_codes.csv (from WorldData).  
First, load the cost of living data into a DF

In [28]:
cost_living_df = pd.read_csv("cost_living_w_codes.csv")
cost_living_df.head(10)

,rank,country_or_region,iso_alpha2,cost_index,monthly_income_USD,purchasing_power_index,notes_special_regions
0,53,Albania,AL,46.3,564 USD,19.2,NaN
1,85,Algeria,DZ,28.8,325 USD,17.7,NaN
2,11,Australia,AU,107.0,"5,036 USD",74.0,NaN
3,26,Austria,AT,83.1,"4,678 USD",88.5,NaN
4,82,Azerbaijan,AZ,31.5,469 USD,23.4,NaN
5,12,Bahamas,BS,105.5,"2,628 USD",39.1,NaN
6,71,Bangladesh,BD,35.8,235 USD,10.3,NaN
7,5,Barbados,BB,123.5,"1,613 USD",20.5,NaN
8,23,Belgium,BE,86.8,"4,058 USD",73.5,NaN
9,81,Benin,BJ,31.5,117 USD,5.8,NaN


Then merge cost of living DF with the current biggie_df

In [29]:
biggie_dfv3 = pd.merge(cost_living_df, biggie_dfv2, on="iso_alpha2", how="left")
biggie_dfv3.head(10)

,rank,country_or_region,iso_alpha2,cost_index,monthly_income_USD,purchasing_power_index,notes_special_regions,country,gender_pay_parity,job_id,...,gbp_converted_25th,gbp_converted_50th,gbp_converted_75th,iso_alpha3,iso_numeric,fips,capital,area_km2,population,continent
0,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ACCOUNT-MANAGER,...,39.462463,49.410083,61.865280,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
1,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ACCOUNTANT,...,31.498831,37.407968,44.425650,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
2,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ADMINISTRATIVE-ASSISTANT,...,24.581887,29.607022,35.659418,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
3,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ARCHITECT,...,53.731278,66.532833,82.384377,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
4,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ATTORNEY,...,42.990049,58.056954,78.404421,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
5,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,BUSINESS-ANALYST,...,43.174036,51.790750,62.127194,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
6,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,BUSINESS-DEVELOPMENT,...,38.930439,51.577568,68.333304,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
7,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,C-LEVEL-EXECUTIVE,...,71.263359,106.843037,160.186589,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
8,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,CASHIER,...,11.407386,15.982880,22.393603,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
9,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,CHEF,...,29.380912,36.089203,44.329141,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU


In [34]:
# Cleaning: the column 'rank' from cost_of_living.csv isn't clear or helpful in a large DF from many sources.  
# rename rank to WD_cost_living_rank
biggie_dfv3.rename(columns={'rank': 'WD_cost_living_rank', 'country_or_region': 'WD_country_or_region'}, inplace=True)
biggie_dfv3.head(10)

,WD_cost_living_rank,WD_country_or_region,iso_alpha2,cost_index,monthly_income_USD,purchasing_power_index,notes_special_regions,country,gender_pay_parity,job_id,...,gbp_converted_25th,gbp_converted_50th,gbp_converted_75th,iso_alpha3,iso_numeric,fips,capital,area_km2,population,continent
0,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ACCOUNT-MANAGER,...,39.462463,49.410083,61.865280,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
1,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ACCOUNTANT,...,31.498831,37.407968,44.425650,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
2,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ADMINISTRATIVE-ASSISTANT,...,24.581887,29.607022,35.659418,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
3,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ARCHITECT,...,53.731278,66.532833,82.384377,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
4,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,ATTORNEY,...,42.990049,58.056954,78.404421,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
5,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,BUSINESS-ANALYST,...,43.174036,51.790750,62.127194,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
6,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,BUSINESS-DEVELOPMENT,...,38.930439,51.577568,68.333304,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
7,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,C-LEVEL-EXECUTIVE,...,71.263359,106.843037,160.186589,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
8,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,CASHIER,...,11.407386,15.982880,22.393603,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU
9,53,Albania,AL,46.3,564 USD,19.2,NaN,Albania,0.791,CHEF,...,29.380912,36.089203,44.329141,ALB,8.0,AL,Tirana,28748.0,2866380.0,EU


### Cleaning: Rename local currency columns to make name shorter and clearer that values are in local currency, from Teleport API

In [ ]:
biggie_dfv3.rename(columns={'salary_percentiles_percentile_25': 'salary_local_25th_pcl', 'salary_percentiles_percentile_50': 'salary_local_50th_pcl', 'salary_percentiles_percentile_75': 'salary_local_75th_pcl'}, inplace=True)
biggie_dfv3.to_csv("sal_country_gender_costliving.csv", index=False) # intermediate backup of DF to .csv file
biggie_dfv3.head(10)

# Code to create csvs from selected columns from the big DF (to help with SQL tables)

In [43]:
# Create reduced DFs to serve as sql table starters
countries_sql_table_df = biggie_dfv3[['iso_alpha2', 'country', 'capital', 'continent', 'area_km2', 'population','gender_pay_parity']].drop_duplicates(subset='iso_alpha2') # excluded 'iso_alpha3', 'iso_numeric' 'fips' 
countries_sql_table_df.to_csv("countries_data_from_biggie_dfv3.csv", index=False)

cost_of_living_sql_table_df = biggie_dfv3[['iso_alpha2', 'WD_cost_living_rank', 'cost_index', 'monthly_income_USD', 'purchasing_power_index']].drop_duplicates(subset='iso_alpha2')
cost_of_living_sql_table_df.to_csv("cost_of_living_data_from_biggie_dfv3.csv", index=False)

salaries_sql_table_df = biggie_dfv3[['iso_alpha2',  'job_id', 'job_title', 'salary_local_25th_pcl', 'salary_local_50th_pcl', 'salary_local_75th_pcl', 'currency_code', 'local_to_gbp_rates','gbp_converted_25th','gbp_converted_50th', 'gbp_converted_75th']]
salaries_sql_table_df.to_csv("salaries_data_from_biggie_dfv3.csv", index=False)

job_sql_table_df = biggie_dfv3[['job_id', 'job_title']].drop_duplicates(subset='job_id')
job_sql_table_df.to_csv("job_data_from_biggie_dfv3.csv", index=False)

# CELLS BEYOND THIS POINT NOT RUN, just notes / ideas!

### Cleaning task. Compare WD_country_or_region side-by-side with country column. If there are no significant differences, lets delete WD_country_or_region

In [37]:
country_cols_df = biggie_dfv3[['WD_country_or_region', 'country']]
country_cols_df.to_csv("compare_country_cols.csv", index=False) # csv file for inspection

### Cleaning task: Delete excess rows which have ended up in our dataset which don't have salary information.
!! Use the local salary columns NOT the gbp_converted columns for the check

Let's identify which rows in our final massive DF don't have corresponding salary info from Teleport.  
These will probably have come from the join with gender_df on the country name 

In [ ]:
biggie_dfv3['salary_percentiles_percentile_50'].isnull().sum()
# 2 country rows in final DF don't have salary data (so weren't in Teleport API of 198 / 252 countries )
# 81	Benin	BJ	31.5	117 USD	5.8
# 14	Vanuatu	VU	102.9	297 USD	4.5

Want to grab the country codes which aren't needed (no salary data), put them in a DF, and export to a csv to keep a record, then delete them from the main df...

In [ ]:
# # Filter rows where 'salary_percentiles_percentile_50' is null
# countries_no_salaries_df = biggie_dfv3[sbiggie_dfv3['salary_percentiles_percentile_50'].isnull()]

# # Export the filtered DataFrame to a CSV file
# countries_no_salaries_df.to_csv('countries_no_salaries.csv', index=False) # should contain 53

# # Drop the filtered rows from the original DataFrame, save as a new dataframe?
# #biggie_dfv4 = biggie_dfv3.dropna(subset=['salary_percentiles_percentile_50'])

### Clearning task: Need to look through biggie_df and look for any odd rows where the country name from gender_pay_parity didn't match the spelling of the country name in salary_countries (it wasn't joined on iso_alpha2 code (it wasn't in gender_pay.csv), it was joined on name which is will have more variation).  
Options: clean the country names before the merge to make sure they match. Or, import from an SQL table (as this will have been properly processed). Or, just manually update them

### Cleaning task: There are a few countries for which we didn't have local>GBP currency conversion rates (a handful). We need to decide whether to manually convert the currencies or whether to delete from the dataset.
Search for "N/A"s in gbp_converted_50th...

### Cleaning task: Need to clean these names, if they are still in the dataset

In [ ]:
CuraÃ§ao	Willemstad
Ã…land	Mariehamn
Saint BarthÃ©lemy	Gustavia
RÃ©union	Saint-Denis
SÃ£o TomÃ© and PrÃ­ncipe

# Define the dodgy characters to search for
dodgy_character = 'Ã' AND .... 

# Create a boolean mask to identify rows with the dodgy character in any field
mask = df.apply(lambda x: x.str.contains(dodgy_character)).any(axis=1)

# Get the rows with the dodgy character
dodgy_rows = df[mask]

# Print or process the dodgy rows
print(dodgy_rows)
